# Goals
- [X] Comparison of country voting alignment with China, over time for all countries, absolute number of votes cast (bubble graph? Geographic heat map?)
- [ ] Comparison of financial assistance received from China, over time for all countries (bubble graph? Geographic heat map?)
      
- [ ] Overall voting alignment with China per vote type/topic, over time for all countries (bubble graph?)

- [ ] Voting alignment with China and financial assistance, over time, per country (2-track bar chart?)
- [ ] Country voting alignment with China pre- and post-vaccine donations, per country (2-track bar chart?)

- [ ] Comparison of country change in voting alignment with financial assistance received (smallest -> largest positive change in voting alignment / amount of financial assistance)
- [ ] Comparison of country change in voting alignment with donated vaccines received (smallest -> largest positive change in voting alignment / vaccines donated)
- [ ] Comparison of change in financial flows to country compared to voting alignment, over time (smallest -> largest increase in flows / smallest -> largest positive change in voting alignment)
      
- [ ] Comparative country Group recipients of Chinese financial aid, over time (donut chart? Bump area chart?)
- [ ] Comparative voting alignment of country Group with China, over time (donut chart? Bump area chart?)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.io import output_notebook, show
from bokeh.models import GeoJSONDataSource, Slider, CustomJS, LinearColorMapper, Range1d
from bokeh.models import HoverTool
from bokeh.plotting import figure
from bokeh.layouts import column, row
import json
from bokeh.layouts import gridplot
from bokeh.models import LinearAxis
%pip install ipywidgets
%pip install colorcet
import ipywidgets as widgets
import colorcet
output_notebook()

In [ ]:
# Load world geometry data
world = gpd.read_file('data/ne_110m_admin_0_countries.shp')

AFR = ['Algeria','Angola','Benin','Botswana,Burkina Faso','Burundi','Cameroon','Congo','Côte d’Ivoire','Democratic Republic of the Congo','Djibouti','Egypt','Eritrea','Ethiopia','Gabon','The Gambia','Ghana','Kenya','Libya','Madagascar','Malawi','Mali','Mauritania','Mauritius','Morocco','Namibia','Nigeria','Rwanda','Senegal','Sierra Leone','South Africa','Somalia','Sudan','Togo','Tunisia','Uganda','Zambia']
APAC = ['Afghanistan','Azerbaijan','Bahrain','Bangladesh','China','Fiji','India','Indonesia','Iraq','Japan','Jordan','Kazakhstan','Kuwait','Kyrgyzstan','Malaysia','Maldives','Marshall Islands','Mongolia','Nepal','Pakistan','Philippines','Qatar','Republic of Korea','Saudi Arabia','Sri Lanka','Thailand','United Arab Emirates','Uzbekistan','Viet Nam']
GRULAC=['Argentina','Bahamas','Bolivia (Plurinational State of)','Brazil','Chile','Costa Rica','Cuba','Ecuador','El Salvador','Guatemala','Honduras','Mexico','Nicaragua','Panama','Paraguay','Peru','Uruguay','Venezuela (Bolivarian Republic of)']
WEOG = ['Australia','Austria','Belgium','Canada','Denmark','Finland','France','Germany','Iceland','Ireland','Italy','Luxembourg','Netherlands','Norway','Portugal','Spain','Switzerland','United Kingdom of Great Britain and Northern Ireland','United States of America']
EG = ['Albania','Armenia','Bosnia and Herzegovina','Bulgaria','Czechia','Croatia','Estonia','Georgia','Hungary','Latvia','Lithuania','Montenegro','Poland','Republic of Moldova','Republic of North Macedonia','Romania','Russian Federation','Slovakia','Slovenia','Ukraine']

# Function to assign color based on country group
def assign_color(country):
    if country in AFR:
        return 'blue'
    elif country in APAC:
        return 'green'
    elif country in GRULAC:
        return 'red'
    elif country in WEOG:
        return 'yellow'
    elif country in EG:
        return 'orange'
    else:
        return 'lightgray'
    
def get_country_group(country):
    if country in AFR:
        return 'AFR'
    elif country in APAC:
        return 'APAC'
    elif country in GRULAC:
        return 'GRULAC'
    elif country in WEOG:
        return 'WEOG'
    elif country in EG:
        return 'EG'
    
def calculate_alignment(row):
    if row["China Vote"] == "Abstaining":
        return 0
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1
    

# Global widgets
fa_type=widgets.Dropdown(
    options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average FA"], description="FA Type:"
)

country_group=widgets.Dropdown(
    options = ["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"], description="Country Group:"
)

fa_colour = {
    "AFR":"blue", "APAC":"green", "GRULAC":"yellow", "WEOG":"orange", "EG":"brown", "All": "grey"
}

## Comparison of country voting alignment with China and Financial Assistance
over time for all countries, absolute number of votes cast

In [ ]:

# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)


china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)



df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_aggregated = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_aggregated.rename(columns={"Text title": "Number of votes"}, inplace=True)
df_aggregated["alignment"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
)
df_aggregated["alignment_percentage"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
) * 100
years = pd.DataFrame({"Year": df["Year"].unique()})
topics = df["Topic"].dropna().unique()

# Load world geometry data (you might need to adjust the file path)
countries = pd.DataFrame({"Country": world["ADMIN"].unique()})

cya = years.merge(countries, how="cross")
cya = cya.merge(df_aggregated, on=["Year", "Country"], how="left")
# Merge your aggregated data with the geopandas DataFrame
merged = world.merge(cya, left_on="ADMIN", right_on="Country", how="left")
merged = merged[
    [
        "featurecla",
        "CONTINENT",
        "REGION_UN",
        "ADMIN",
        "SUBREGION",
        "REGION_WB",
        "MIN_ZOOM",
        "MIN_LABEL",
        "MAX_LABEL",
        "LABEL_X",
        "LABEL_Y",
        "geometry",
        "Year",
        "Country",
        "alignment_score",
        "Number of votes",
        "alignment",
        "alignment_percentage",
        "FT_ODI",
        "AEI_FDI",
        "BU_ODA",
        "ODI_MOFCOM",
    ]
]
merged["Average FA"] = merged[["FT_ODI","AEI_FDI","BU_ODA","ODI_MOFCOM"]].mean(axis=1, skipna=True)
merged_initial = merged[(merged["Year"] == 2006)]
# Convert to GeoJSON
geojson = json.dumps(merged_initial.__geo_interface__)
geojson_original = json.dumps(merged.__geo_interface__)
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson=geojson)

# Determine the bounds for your map
x_range = Range1d(start=-180, end=180)
y_range = Range1d(start=-90, end=90)


def render_layout(fa_type):
    tools = "wheel_zoom,pan,reset"
    va_map = figure(
        title="Voting Alignment with China",
        height=600,
        width=950,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )

    # Define color mapper
    color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38, low=100, high=-100
    )

    # Add patch renderer to figure
    va_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": "alignment_percentage", "transform": color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
    ]

    va_map.add_tools(HoverTool(tooltips=tooltips))

    slider = Slider(
        start=df["Year"].min(),
        end=df["Year"].max(),
        value=df["Year"].min(),
        step=1,
        title="Year",
    )
    callback = CustomJS(
        args=dict(source=geosource, available=geojson_original, slider=slider),
        code="""
        const year = slider.value;
        const geojson = (JSON.parse(source.geojson))
        const features = JSON.parse(available).features.filter(feature => feature.properties.Year === year);
        const new_geojson = {type:geojson.type, features:features}

        source.geojson = JSON.stringify(new_geojson);
        source.change.emit();
    """,
    )
    slider.js_on_change("value", callback)
    fa_color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38,
        low=0,
        high=df_aggregated[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]]
        .max(axis=1)
        .max(),
    )
    fa_map = figure(
        title="Financial Assistance from China",
        height=600,
        width=950,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )
    fa_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": fa_type, "transform": fa_color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
        ("Financial Assistance", f"@{fa_type}"),
        ("Financial Assistance type", fa_type),
    ]
    fa_map.add_tools(HoverTool(tooltips=tooltips))
    layout = column(slider, row(va_map, fa_map))
    show(layout)


widgets.interact(
    render_layout,
    fa_type=fa_type,
)

## Country voting alignment % vs FA by FA type
over time for all countries

In [ ]:
# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)

# Create a new column 'vote_nr' based on the values in the 'Vote' column

china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
df["Country Group"] = df["Country"].apply(get_country_group)

# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)

df_fa = df.groupby(["Year", "Country"]).agg({"FT_ODI": "first",
        "AEI_FDI":"first",
        "BU_ODA":"first",
        "ODI_MOFCOM":"first"})

df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_aggregated = (
    df.groupby(["Year", "Country", "Country Group"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
        }
    )
    .reset_index()
)
df_aggregated.rename(columns={"Text title": "Number of votes"}, inplace=True)
df_aggregated["alignment"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
)
df_aggregated["alignment_percentage"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
) * 100
df_aggregated = df_aggregated.merge(df_fa, on=["Year", "Country"], how="left")
# Load world geometry data (you might need to adjust the file path)
years = pd.DataFrame({"Year": df["Year"].unique()})
countries = pd.DataFrame({"Country": world["ADMIN"].unique()})

cya = years.merge(countries, how="cross")
cya = cya.merge(df_aggregated, on=["Year", "Country"], how="left")
# Merge your aggregated data with the geopandas DataFrame
cya = cya[
    [
        "Year",
        "Country",
        "Country Group",
        "ODI_MOFCOM",
        "BU_ODA",
        "AEI_FDI",
        "FT_ODI",
        "alignment_score",
        "Number of votes",
        "alignment",
        "alignment_percentage",
    ]
]
cya["Average FA"] = cya[["ODI_MOFCOM","BU_ODA","AEI_FDI","FT_ODI"]].mean(skipna=True,axis=1)
cya.dropna()

def plot(fa_type, country_group, nr_plots):
    list_plots = []
    if country_group=="All":
        filtered_df = cya
    else:
        filtered_df = cya[cya["Country Group"]==country_group]
    for country in filtered_df["Country"].unique().tolist():
        temp = filtered_df[filtered_df["Country"] == country]
        if temp[fa_type].any():
            x=temp["Year"].to_numpy()
            plot = figure(width=450, height=350, title=country)
            plot.y_range = Range1d(start=-100, end=+100)
            plot.extra_y_ranges = {"FA": Range1d(start=0, end=temp[fa_type].max())}
            plot.add_layout(LinearAxis(y_range_name="FA", axis_label="Financial Assistance"), "right")
            plot.circle(x=x, y=temp["alignment_percentage"].tolist())
            ap_par = np.polyfit(x, temp["alignment_percentage"].to_numpy(), 1, full=True)
            ap_slope=ap_par[0][0]
            ap_intercept=ap_par[0][1]
            ap_y_predicted = [ap_slope*i + ap_intercept  for i in x]
            plot.line(x, ap_y_predicted, color="blue", legend_label='y='+str(round(ap_slope,2))+'x+'+str(round(ap_intercept,2)))          
            plot.square(x=x, y=temp[fa_type], y_range_name="FA", fill_color="red", color="red")
            plot.line(x=x, y=temp[fa_type], y_range_name="FA", line_color="red", legend_label="Financial Assitance")
            list_plots.append(plot)
    x = nr_plots
    grid = [list_plots[i : i + x] for i in range(0, len(list_plots), x)]
    p = gridplot(grid)
    show(p)


widgets.interact(
    plot,
    fa_type=fa_type,
    country_group=country_group,
    nr_plots=widgets.BoundedIntText(
        value=4,
        min=2,
        max=10,
        step=1,
        description="Plots (2 to 10):",
        disabled=False,
    ),
)


In [ ]:

# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)

# Create a new column 'vote_nr' based on the values in the 'Vote' column

china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)
df["Country Group"] = df["Country"].apply(get_country_group)


df_fa = df.groupby(["Year", "Country", "Country Group"]).agg(
    {"FT_ODI": "first", "AEI_FDI": "first", "BU_ODA": "first", "ODI_MOFCOM": "first"}
)
df_fa = df_fa.groupby(["Year", "Country Group"]).agg(
     {"FT_ODI": "sum", "AEI_FDI": "sum", "BU_ODA": "sum", "ODI_MOFCOM": "sum"}
)
df_fa["Average FA"] = df_fa[["ODI_MOFCOM","BU_ODA","AEI_FDI","FT_ODI"]].replace(0, np.nan).mean(axis=1, skipna=True)
df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_aggregated = (
    df.groupby(["Year", "Country Group"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
        }
    ).reset_index()
)
df_aggregated.rename(columns={"Text title": "Number of votes"}, inplace=True)
df_aggregated["alignment"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
)
df_aggregated["alignment_percentage"] = (
    df_aggregated["alignment_score"] / df_aggregated["Number of votes"]
) * 100
df_aggregated = df_aggregated.merge(df_fa, on=["Year", "Country Group"], how="left")
# Load world geometry data (you might need to adjust the file path)


def plot(fa_type, country_group):
    plot=None
    if country_group=="All":
        filtered_df = df_aggregated
    else:
        filtered_df = df_aggregated[df_aggregated["Country Group"]==country_group]
    plot = figure(width=950, height=600, title="Overall FA vs Alignment")
    plot.y_range = Range1d(start=-100, end=+100)
    plot.extra_y_ranges = {"FA": Range1d(start=0, end=filtered_df[fa_type].max())}
    plot.add_layout(
        LinearAxis(y_range_name="FA", axis_label="Financial Assistance"),
        "right",
    )
    x = filtered_df["Year"].to_numpy()
    plot.vbar(
        x=x,
        top=filtered_df[fa_type],
        width=0.9,
        y_range_name="FA",
        fill_color="red",
        legend_label=f"Financial assistance:{fa_type}"
    )
    ap_par = np.polyfit(x, filtered_df["alignment_percentage"].to_numpy(), 1, full=True)
    ap_slope=ap_par[0][0]
    ap_intercept=ap_par[0][1]
    ap_y_predicted = [ap_slope*i + ap_intercept  for i in x]
    plot.line(x, ap_y_predicted, color="blue", legend_label='y='+str(round(ap_slope,2))+'x+'+str(round(ap_intercept,2)))
    plot.circle(
        x=x,
        y=filtered_df["alignment_percentage"].tolist(),
        legend_label="Alignment %"
    )

    show(plot)




widgets.interact(
    plot,
    fa_type=fa_type,
    country_group=country_group
)
df_aggregated.head()

## Comparison of financial assistance received from Chinaover time for all countries